In [2]:
import random as rd
import numpy as np
from math import sqrt, floor, log2

Implémentation de RSA sous forme de classe.


In [69]:
class RSA:
    
    # Huge prime number
    _PRIME = 4879228431756730165299385010232837291120885737358336711467496639025522618199839237653918283371408014017658368720147197955013865183708637800199520868317779
    def __init__(self, n, k0, e):
        self.n = n
        self.k0 = k0
        self.e = e
        pass
    
    @staticmethod
    def getPrime(): return RSA._PRIME
    
    @staticmethod
    def bigPrime(l : int) -> int:
        """bigPrime Génère un nombre premier de 'l' bits
        
        Args:
            l (int): Taille en bit du nombre
        
        Returns:
            int: Nombre premier sur l bits 
        """

        # Génère un nombre aléatoire de l-bits (impair)
        n = rd.getrandbits(l) | 1
        n |= (1 << l - 1)

        # Boucle pour s'assurer que n est premier
        while not RSA.millerRabin(n, 5):
            # Génère un nombre aléatoire de l-bit et fait un OR avec 1
            n = rd.getrandbits(l) | 1   # Obligatoirement impair
            n |= (1 << l - 1)
        return n
    
    @staticmethod
    def SnM(a , b , n : int ): 
        if (type(a) == str):
            a = int(a, 2)
        if (type(b) == str):
            b = int(a, 2)
        return pow(a, b, n)
    
    @staticmethod
    def EA(a : int, b : int) -> int:
        """
        EEA Méthode pour déterminer le PGCD de deux nombre entier
        
        Args:
            a (int): Premier nombre entier
            b (int): Second nombre entier
        
        Returns:
            int : Plus grand commun diviseur des nombre a et b
        """

        # Cas de base
        if (b == 0): return a
        if (a == b): return b
        
        # Appel récursif selon le cas 
        if (a > b): return RSA.EA(b, a%b)
        else : return RSA.EA(a, b%a)
    
    @staticmethod
    def EEA(a : int, b : int) -> int:
        """EEA Méthode pour déterminer le PGCD de deux nombres entiers
        
        Args:
            a (int): Premier nombre entier
            b (int): Second nombre entier
        
        Returns:
            int[]: Coefficients et plus grand commun diviseur des nombres a et b
        """
        # S'assure que le nombre 'a' est plus grand
        if (a < b): RSA.EEA(b, a)

        # Initialisation des coefficients := [[x0, x1], [y0, y1]]
        coeff = [[1, 0], [0, 1]]

        # Initialise un tableau pour contenir les coefficients 'x' et 'y' ainsi que le PGCD
        result = [None, None, None]

        # Boucle principale
        while(True) :
            # Détermine q et r
            r = a % b   
            if (r == 0): return result
            q = a // b 

            # Obtient les résultats de la ie itération
            result[0] = coeff[0][0] - q * coeff[0][1]
            result[1] = coeff[1][0] - q * coeff[1][1]
            result[2] = r

            # Modifie les coefficients 
            coeff[0] = [coeff[0][1], result[0]]
            coeff[1] = [coeff[1][1], result[1]]

            # Modifie les valerus de 'a' et 'b' qui seront utilisées pour la prochaine itération
            a = b
            b = r
    
    @staticmethod
    def hash_function(m : int, l : int):
        """hash_function Retourne un hash de longueur 'l' de 'm'
        
        Args:
            m (int): Message à hasher
            l (int): Longueur du hash 
        """
        if (type(m) == str):
            m = int(m, 2)
        # Convertie le message en nombre
        mhash = bin((m * RSA.getPrime()) % pow(2, l-1))
        return mhash[2:].zfill(l)

    

    @staticmethod
    def millerRabin(n : int, s : int) -> bool:
        """millerRabin Test si un nombre est premier
        
        Args:
            n (int): Nombre à tester
            s (int): Nombre de test
        
        Returns:
            bool: Vrai si n est premier
        """
        # Vérifie si le nombre 'p' est pair; retourne vrai si 'p' vaut 2 et faux sinon
        if (n % 2 == 0) : return n == 2
        if n == 3: return True

        # Initialise r, u et _p
        u, _n = 0, n-1
        # Détermine la valeur de 'u'
        while _n & 1 == 0 :
            u += 1      # Augmente la valeur de 'u'
            _n //= 2    # Division entière de _n par 2

        # Boucle principale
        for i in range(s):
            # Détermine un a aléatoire [2, n-1[
            a = rd.randrange(2, n-1)
            # Détermine 'z' initiale
            z = pow(a, _n, n)

            if (z != 1 and z != n-1): 
                # Boucle pour déterminer si 'n' est composé
                j = 1
                while j < u and z != n-1:
                    # Nouvelle valeur de z
                    z = pow(z, _n, n)
                    if z == 1: return False
                    # Augmente le compteur
                    j += 1
                if z != n-1: return False
        return True
    
    #staticmethod
    def OAEP(self, msg : str) -> str:
        """OAEP Méthode implémentant «l'Optimal Asymmetric Encryption Padding»
        
        Args:
            msg (str): Message à «padder»
            l (int)  : Longueur totale du message «paddé»
        
        Returns:
            str: Message «paddé»
        """
        k0 = self.k0
        n = self.n
        k1 = n - k0 - len(msg)
        # Genere un padding
        padding = "0" * k1
        # Ajoute le padding
        m = msg + padding
        
        # Génère un nombre aléatoire
        r = bin(rd.randrange(0,pow(2, k0-1)))[2:]
        r = r.zfill(k0)
        
        # Hash 'r' à n-k0 bits
        hash_r = RSA.hash_function(r, n-k0)
        
        # X == m XOR hash_r
        X = ''.join(str(int(a) ^ int(b)) for a,b in zip(m, hash_r))
        # Hash 'X'
        hash_X = RSA.hash_function(X, k0)

        # Y == r XOR hash_X
        Y = ''.join(str(int(a) ^ int(b)) for a,b in zip(r, hash_X))
        
        return X + Y
    
    #@staticmethod
    def OAEP_inv(self, XY : str) -> str:
        """OAEP_inv Méthode inverse de «OAEP» et permet de retrouver 'm'
        
        Args:
            XY (str): Message «paddé»
        
        Returns:
            str : Meesage original
        """
        k0 = self.k0
        n = self.n
        # Sépare le message en deux parties: X || Y
        X = XY[:n-k0]
        Y = XY[n-k0:]
        
        # Hash 'X'
        hash_X = RSA.hash_function(X, k0)#[2:].zfill(k0)
        # Retrouve 'r'
        r = ''.join(str(int(a)^int(b)) for a, b in zip(Y, hash_X))
        # Hash 'r'
        hash_r = RSA.hash_function(r, n-k0)#[2:].zfill(l-k0)
        # Retrouve 'msg + padding'
        m = ''.join(str(int(a)^int(b)) for a, b in zip(X, hash_r))
        
        return m
    
    #@staticmethod
    def genKeys(self, l = None):
        """genKeys Méthode pour générer les clés 'PK' et 'SK'
        
        Args: 
            l (int): Longueur de 'n'
            
        Returns:
            tuple : Clés PK et SK
        """
        # Détermine la valeur de 'l'
        if l is None: l = self.n
            
        # Obtient la valeur de 'e'
        e = self.e
        # Initialise p et q
        p, q = e+1, e+1
        
        # Obtient les bonnes valeurs de q et p
        while (p-1)%e == 0: p = RSA.bigPrime(l//2)
        while (q-1)%e == 0: q = RSA.bigPrime(l//2)
        
        # Détermine la valeur de 'n' et 'phi_n'
        n = p * q
        phi_n = (p-1) * (q-1)
        
        print(RSA.EEA(e, phi_n))
        
        # Détermine la valeur de 'd'
        d = RSA.EEA(e, phi_n)[0]
        if d < 0: d += phi_n-1
        print("d*e:", (d*e)%phi_n)
        #d = RSA.SnM(e, phi_n-1, n)
        
        
        # Forme les clés
        PK = (n, e)
        SK = (p, q, d)
        
        return PK, SK

    #@staticmethod
    def exp_CRT(self, C : str, SK : list):
        # Convertie 'C' en valeur numérique
        C_num = int(C, 2)
        
        # Obtient les valeurs p, q et d de la clé SK
        p, q, d = SK[0], SK[1], SK[2]
        # Obtient la taille
        n = self.n
        
        dp, dq = d%(p-1), d%(q-1)
        
        res = RSA.EEA(q, p)
        dInv = res[0]
        if dInv < 0: dInv += p
        
        m1 = RSA.SnM(C, dp, p)
        m2 = RSA.SnM(C, dq, q)
        h = (dInv * (m1 - m2))%p
        m = m2 + h * q
        m = bin(m)[2:]
        
        
        kp, kq = RSA.SnM(q, p-1, p), RSA.SnM(p, q-1, q)
        xp, xq = C_num%p, C_num%q
        yp, yq = RSA.SnM(xp, dp, p), RSA.SnM(xq, dq, q)
        
        # Obtient le message
        msg = bin(((q*kp)*yp + (p*kq)*yq)%(p*q))[2:].zfill(n)
        return msg
         
    #self, @staticmethod
    def encrypt(self, msg : str, PK : list):
        """encrypt Méthode pour encrypter un message avec une clé publique
        
        Args:
            msg (str): Message à encrypter
            PK (list): Clé publique
            
        Returns:
            cipher (str): Message encrypté
        """
        # Obtient 'n' et 'e' de la clé publique
        N, e = PK[0], PK[1]
        # Obtient la taille
        n = self.n
        # Initialise le cipher
        cipher = ""
        
        block = ""
        # Encrypte le message par block de n-bits
        for i in range(len(msg)//n):
            block = msg[i*n : (i+1)*n]
            cipher += bin(RSA.SnM(block, e, N))[2:].zfill(n)
        
        # Obtient la longueur du message
        length = len(msg)
        # Convertie la taille en binaire
        msg_info = bin(length)[2:].zfill(n-1)
        
        # Détermine la taille du restant
        reminder_size = len(msg)%n
        # Si le restant est de taille nulle, alors ajoute un '0'
        if reminder_size == 0:
            # Place un marqueur
            msg_info = '0' + msg_info
        elif (reminder_size <= self.n-self.k0):
            # Place un marqueur
            msg_info = '1' + msg_info
            # Padding
            cipher += bin(RSA.SnM(self.OAEP(msg[-reminder_size:]), e, N))[2:].zfill(n)
        else:
            # Place un marquer
            msg_info = '0' + msg_info
            # Encrypte le restant
            block = msg[-reminder_size:] + "0" * (n-reminder_size)
            cipher += bin(RSA.SnM(block, e, N))[2:].zfill(n)
            
        # Encrypte l'information du message
        msg_info_enc = bin(RSA.SnM(msg_info, e, N))[2:].zfill(n)
        # Ajoute l'information au début du cipher
        print("Original:", msg_info, block)
        print("Encrypté:", msg_info_enc, cipher)
        return msg_info_enc + cipher
    
    #@staticmethod
    def decrypt(self, cipher : str, SK : list):
        """decrypt Méthode pour décrypter un cipher selon une clé privée
        
        Args:
            cipher (str): Meesage à décrypter
            SK (list): Clé privée
        
        Returns:
            msg (str): Message décrypté
        """
        # Taille des blocks encryptés
        n = self.n#SK[0] * SK[1]
        # Détermine le nombre de block à décrypter
        nblock = len(cipher) // n
        # Initialise le message
        msg = ""
        
        # Obtient le premier block (contient les infos sur le message)
        msg_info_enc = cipher[0:n]
        print("msg_info_enc:", msg_info_enc)
        # Décrypte le block
        #msg_info = pow(int(msg_info_enc, 2), SK[2], n)
        msg_info = self.exp_CRT(msg_info_enc, SK)
        print("msg_info:    ", msg_info)
        #msg_info = bin(msg_info)[2:].zfill(n)
        # Détermine si un padding a été fait
        padded = (msg_info[0] == "1")
        
        # Obtient la taille du message
        index = 1;
        while msg_info[index] == '0': 
            index += 1
            if index == len(msg_info): break
                
        if index < len(msg_info):
            length_bin = msg_info[index:]
        else:
            length_bin = "0"
            
        print("length_bin:",length_bin)
        length = int(length_bin, 2)
        print("length:",length)
            
        
        # Boucle pour décrypter les blocks
        for i in range(1, nblock-1):
            block = cipher[i*n: (i+1)*n]
            msg += self.exp_CRT(block, SK)
            
        block = cipher[-n:]
        if padded:
            block = self.OAEP_inv(block)[0:length%n]
        msg += self.exp_CRT(block, SK)[0:length%n]
        
        return msg


Test de la classe

In [70]:
N = 1000000
length = 64

res = RSA.EEA(240, 17)
x = res[0]
if x < 0:
    x += 17
print(x)

msg = "010100001000001001000100"
print(len(msg))

rsa = RSA(32, 12, 257)
PK, SK = rsa.genKeys()

cipher = rsa.encrypt(msg, PK)

msg_dec = rsa.decrypt(cipher, SK)
print(msg_dec)







        


9
24
[431958821, -43, 1]
d*e: 1
Original: 00000000000000000000000000011000 01010000100000100100010000000000
Encrypté: 00010000000000111100011010111110 01001001001110101001100101110001
msg_info_enc: 00010000000000111100011010111110
msg_info:     00000000001001010100110001110000
length_bin: 1001010100110001110000
length: 2444400
0100001110110011
